In [ ]:
data_train_dir = "/content/insectos_corregido"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/insectos_corregido.zip"

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: insectos_corregido/8335452/d075s0348.jpg  
  inflating: insectos_corregido/8335452/d075s0349.jpg  
  inflating: insectos_corregido/8335452/d075s0350.jpg  
  inflating: insectos_corregido/8335452/d075s0351.jpg  
  inflating: insectos_corregido/8335452/d075s0352.jpg  
  inflating: insectos_corregido/8335452/d075s0364.jpg  
  inflating: insectos_corregido/8335452/d075s0365.jpg  
  inflating: insectos_corregido/8335452/d075s0366.jpg  
  inflating: insectos_corregido/8335452/d075s0367.jpg  
  inflating: insectos_corregido/8335452/d075s0368.jpg  
  inflating: insectos_corregido/8335452/d075s0369.jpg  
  inflating: insectos_corregido/8335452/d075s0370.jpg  
  inflating: insectos_corregido/8335452/d075s0383.jpg  
  inflating: insectos_corregido/8335452/d075s0384.jpg  
  inflating: insectos_corregido/8335452/d075s0385.jpg  
  inflating: insectos_corregido/8335452/d075s0386.jpg  
  inflating: insectos_corregido/8335452/d07

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, Input, Add
from keras.layers import BatchNormalization as BN
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.metrics import TopKCategoricalAccuracy

In [ ]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range = 0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [ ]:
target_size = (64, 32)
batch_size = 512
mode = 'categorical'
numclasses = 291

In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=mode,
    subset='training') # set as training data

Found 50776 images belonging to 291 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    data_train_dir, # same directory as training data
    target_size=target_size,
    batch_size=batch_size,
    class_mode=mode,
    subset='validation') # set as validation data

Found 12543 images belonging to 291 classes.


In [ ]:
inputs = Input((*target_size, 3))

In [ ]:
## DEF A RESNET BLOCK CONV + BN + GN + MAXPOOL
def ResNetCBGN(x,filters):

  y = Conv2D(filters = filters, kernel_size = 3, padding='same')(x) 
  y = BN()(y)
  y = Activation('relu')(y)
  y = Conv2D(filters = filters, kernel_size=  3, padding='same')(y)
  y = BN()(y)
  y = Activation('relu')(y)
  y = Conv2D(filters = filters, kernel_size=  3, padding='same')(y)
  y = BN()(y)

  x = Conv2D(filters = filters, kernel_size=  1, padding='same')(x)
  x = BN()(x)
  out = Add()([x, y])

  out= BN()(out)
  out = Activation('relu')(out)
  out = MaxPooling2D(pool_size=(2, 2))(out)
  
  return out

In [ ]:
## DEF RESNET TOPOLOGY
filters = 32
resNetModel = ResNetCBGN(inputs,filters)
filters = 64
resNetModel = ResNetCBGN(resNetModel,filters)
filters = 128
resNetModel = ResNetCBGN(resNetModel,filters)
filters = 256
resNetModel = ResNetCBGN(resNetModel,filters)
filters = 512
resNetModel = ResNetCBGN(resNetModel,filters)

resNetModel = Flatten()(resNetModel)
resNetModel = Dense(512)(resNetModel)
resNetModel = Activation('relu')(resNetModel)
resNetModel = Dense(numclasses)(resNetModel)
outputs = Activation('softmax')(resNetModel)

modelResFinal = Model(inputs, outputs)
modelResFinal.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 32, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 32, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 32, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.00001)

In [ ]:
## OPTIM AND COMPILE

modelResFinal.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', TopKCategoricalAccuracy(k=3, name="Top_3_accuracy")])

In [ ]:

modelResFinal.load_weights("/content/drive/MyDrive/Colab Notebooks/insectos_mi_red_resnet/insectos_mi_red_resnet_weight")
evaluating_model = modelResFinal.evaluate_generator(validation_generator, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 66s 2s/step - loss: 0.8453 - accuracy: 0.7995 - Top_3_accuracy: 0.9273


In [ ]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(x,filters):
  y = Conv2D(filters = filters, kernel_size = 3, padding='same')(x) 
  y = BN()(y)
  y = Activation('relu')(y)
  y = Conv2D(filters = filters, kernel_size=  3, padding='same')(y)
  y = BN()(y)
  y = Activation('relu')(y)
  y = Conv2D(filters = filters, kernel_size=  3, padding='same')(y)
  y = BN()(y)
  y = Activation('relu')(y)
  out = MaxPooling2D(pool_size=(2, 2))(y)
  
  return out

In [ ]:
## DEF NN TOPOLOGY  
filters = 32
model_propio = CBGN(inputs,filters)
filters = 64
model_propio = CBGN(model_propio,filters)
filters = 128
model_propio = CBGN(model_propio,filters)
filters = 256
model_propio = CBGN(model_propio,filters)
filters = 512
model_propio = CBGN(model_propio,filters)

model_propio = Flatten()(model_propio)
model_propio = Dense(512)(model_propio)
model_propio = Activation('relu')(model_propio)
model_propio = Dense(numclasses)(model_propio)
outputs = Activation('softmax')(model_propio)

model_propio = Model(inputs, outputs)
model_propio.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 32, 3)]       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 64, 32, 32)        896       
_________________________________________________________________
batch_normalization_25 (Batc (None, 64, 32, 32)        128       
_________________________________________________________________
activation_17 (Activation)   (None, 64, 32, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 64, 32, 32)        9248      
_________________________________________________________________
batch_normalization_26 (Batc (None, 64, 32, 32)        128       
_________________________________________________________________
activation_18 (Activation)   (None, 64, 32, 32)        0   

In [ ]:

model_propio.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', TopKCategoricalAccuracy(k=3, name="Top_3_accuracy")])

In [ ]:
model_propio.load_weights("/content/drive/MyDrive/Colab Notebooks/insectos_mi_red/insectos_mi_red_weight")
evaluating_model = model_propio.evaluate_generator(validation_generator, verbose=1)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc3183f98> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f5bc5913eb8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f5bc3213a90> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc3183f98>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc319ab38> and <tensorflow.python.keras.layers.core.Activation object at 0x7f5bc3183c50>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f5bc318bda0> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc319ab38>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.la

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 56s 2s/step - loss: 0.8466 - accuracy: 0.7662 - Top_3_accuracy: 0.9196


In [ ]:
## DEF NN TOPOLOGY  
model_paper = Conv2D(filters = 32, kernel_size = (3,3))(inputs) 
model_paper = BN()(model_paper)
model_paper = Activation('relu')(model_paper)
model_paper = Conv2D(filters = 32, kernel_size = (3,3))(model_paper)
model_paper = BN()(model_paper)
model_paper = Activation('relu')(model_paper)
model_paper = MaxPooling2D(pool_size=(2, 2))(model_paper)

model_paper = Conv2D(filters = 64, kernel_size = (3,3))(model_paper)
model_paper = BN()(model_paper)
model_paper = Activation('relu')(model_paper)
model_paper = MaxPooling2D(pool_size=(2, 2))(model_paper)

model_paper = Conv2D(filters = 128, kernel_size = (3,3))(model_paper)
model_paper = BN()(model_paper)
model_paper = Activation('relu')(model_paper)
model_paper = Conv2D(filters = 128, kernel_size = (3,3))(model_paper)
model_paper = BN()(model_paper)
model_paper = Activation('relu')(model_paper)
model_paper = MaxPooling2D(pool_size=(2, 2))(model_paper)


model_paper = Flatten()(model_paper)
model_paper = Dense(128)(model_paper)
model_paper = Activation('relu')(model_paper)
model_paper = Dense(numclasses)(model_paper)
outputs = Activation('softmax')(model_paper)

model_paper = Model(inputs, outputs)
model_paper.summary()


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 32, 3)]       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 62, 30, 32)        896       
_________________________________________________________________
batch_normalization_55 (Batc (None, 62, 30, 32)        128       
_________________________________________________________________
activation_55 (Activation)   (None, 62, 30, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 60, 28, 32)        9248      
_________________________________________________________________
batch_normalization_56 (Batc (None, 60, 28, 32)        128       
_________________________________________________________________
activation_56 (Activation)   (None, 60, 28, 32)        0   

In [ ]:
model_paper.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', TopKCategoricalAccuracy(k=3, name="Top_3_accuracy")])

In [ ]:
model_paper.load_weights("/content/drive/MyDrive/Colab Notebooks/insectos_paper/insectos_paper_weight")
evaluating_model = model_paper.evaluate_generator(validation_generator, verbose=1)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc03ece48> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f5bc5913eb8>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f5bc4de3898> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc03ece48>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc4dad160> and <tensorflow.python.keras.layers.core.Activation object at 0x7f5bc02b3320>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f5bc0379cf8> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5bc4dad160>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.la

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 56s 2s/step - loss: 1.4228 - accuracy: 0.6069 - Top_3_accuracy: 0.8096


In [ ]:
from keras.layers import Average
models = [model_propio, modelResFinal, model_paper]

    
outputs = [model.outputs[0] for model in models]
y = Average()(outputs)
model_ensembled = Model(inputs, y, name='ensembled')


In [ ]:
model_ensembled.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', TopKCategoricalAccuracy(k=3, name="Top_3_accuracy")])
model_ensembled.evaluate_generator(validation_generator, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 58s 2s/step - loss: 0.7521 - accuracy: 0.8099 - Top_3_accuracy: 0.9292


[0.7553880214691162, 0.8091365694999695, 0.9268117547035217]